In [1]:
from selenium.webdriver.chrome.options import Options
from os import path, listdir, rename, getcwd
from fake_useragent import UserAgent
from dateutil.parser import parse
from selenium import webdriver
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import pickle
import time
import re

In [2]:
def chrome_user_agent():
    return UserAgent().chrome

chrome_user_agent()

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36 Mozilla/5.0 (iPad; U; CPU OS 3_2 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B334b Safari/531.21.10'

In [3]:
def chrome_driver():
    options = Options()
    options.add_argument('--headless')
    options.add_argument("window-size=1600,900")
    options.add_argument(f'user-agent={chrome_user_agent()}')
        
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")

    return webdriver.Chrome(options=options)

In [4]:
def get_soup(url):
    driver.get(url)
    time.sleep(2)
    return BeautifulSoup(driver.page_source, 'html.parser')

In [5]:
def check_products(soup, category, subcategory, drops=[]):
    sku_list = soup.find('ol', {'class', 'sku-item-list'}).findAll('li', {'class', 'sku-item'})

    for sku in sku_list:
        title = sku.find('div', {'class', 'sku-title'}).find('a')

        if re.search("combo", title['href']):
            if sku.find('div', {'class', 'combo-add-to-cart-button'}).find('svg'):
                link = sku.find('div', {'class', 'sku-title'}).find('a')['href']
                id = link[-12:]
                name = title.text.strip()
                price = sku.find('div', {'class', 'priceView-customer-price'}).find('span').text[1:]
                button = sku.find('button', {'class', 'add-to-cart-button'}).text.strip()

                drops.append([id, name, 2, category, subcategory, price, link, datetime.now().strftime('%Y/%m/%d %H:%M:%S'), '', True])
        else:
            if sku.find('button', {'class', 'add-to-cart-button'}).find('svg'):
                id = sku.find('div', {'class', 'sku-title'}).find('a')['href'][-7:]
                name = title.text.strip()
                price = sku.find('div', {'class', 'priceView-customer-price'}).find('span').text[1:]
                button = sku.find('div', {'class', 'sku-list-item-button'}).find('button').text.strip()
                link = f'https://api.bestbuy.com/click/-/{id}/pdp'
                atc = f'https://api.bestbuy.com/click/-/{id}/cart'

                drops.append([id, name, 2, category, subcategory, price, link, datetime.now().strftime('%Y/%m/%d %H:%M:%S'), atc, False])

    if soup.find('div', {'class', 'footer-category'}).find_next_sibling('div') and not soup.find('a', {'class', 'sku-list-page-next disabled'}):
        driver.get(soup.find('a', {'class', 'sku-list-page-next'})['href'])
        time.sleep(2)
        return check_products(BeautifulSoup(driver.page_source, 'html.parser'), category, subcategory, drops)
    else:
        return drops

In [6]:
def check_product_page(soup, id, category, subcategory, drops=[]):
    if soup.find('button', {'class', 'btn btn-primary btn-lg btn-block btn-leading-ficon add-to-cart-button'}):
        name = soup.find('div', {'class', 'sku-title'}).find('h1').text.strip()
        price = soup.find('div', {'class', 'priceView-customer-price'}).find('span').text[1:]
        link = f'https://api.bestbuy.com/click/-/{id}/pdp'
        atc = f'https://api.bestbuy.com/click/-/{id}/cart'

        drops.append([id, name, 2, category, subcategory, price, link, datetime.now().strftime('%Y/%m/%d %H:%M:%S'), atc, False])
    return drops

In [7]:
def check_xbox_consoles(drops):
    xbox_url = 'https://www.bestbuy.com/site/xbox-series-x-and-s/xbox-series-x-and-s-consoles/pcmcat1586900952752.c?id=pcmcat1586900952752'
    return check_products(get_soup(xbox_url), 3, 1, drops)

In [8]:
def check_ps5_consoles(drops):
    ps5_url = 'https://www.bestbuy.com/site/playstation-5/ps5-consoles/pcmcat1587395025973.c?id=pcmcat1587395025973'
    return check_products(get_soup(ps5_url), 3, 2, drops)

In [9]:
def check_nintendo_switches(drops):
    nintendo_url = 'https://www.bestbuy.com/site/nintendo-switch/nintendo-switch-consoles/pcmcat1484077694025.c?id=pcmcat1484077694025'
    return check_products(get_soup(nintendo_url), 3, 3, drops)

In [10]:
def check_rtx_3000_series(drops):
    rtx_dict = {
        '3060': 4,
        '3060 Ti': 5,
        '3070': 6,
        '3080': 7,
        '3090': 8
        }

    rtx_url = 'https://www.bestbuy.com/site/computer-cards-components/video-graphics-cards/abcat0507002.c?id=abcat0507002&qp=gpusv_facet%3DGraphics%20Processing%20Unit%20(GPU)~NVIDIA%20GeForce%20RTX%20{gpu}'

    for gpu in rtx_dict:
        drops = check_products(get_soup(rtx_url.format(gpu=gpu).replace(' ','%20')), 1, rtx_dict[gpu], drops)
    return drops

In [11]:
def check_radeon_rx_series(drops):
    radeon_dict = {
        '6700 XT': 9,
        '6800 XT': 10,
        '6800': 11,
        '6900 XT': 12
        }

    radeon_url = 'https://www.bestbuy.com/site/computer-cards-components/video-graphics-cards/abcat0507002.c?id=abcat0507002&qp=gpusv_facet%3DGraphics%20Processing%20Unit%20(GPU)~AMD%20Radeon%20RX%20{gpu}'

    for gpu in radeon_dict:
        drops = check_products(get_soup(radeon_url.format(gpu=gpu).replace(' ','%20')), 1, radeon_dict[gpu], drops)
    return drops

In [12]:
def check_ryzen_series(drops=[]):
    ryzen_dict = {
        '5600x': ['6438943', 13],
        '5800X': ['6439000', 14],
        '5900X': ['6438942', 15],
        '5950X': ['6438941', 16]
    }

    product_url = 'https://api.bestbuy.com/click/-/{product}/pdp'

    for gpu in ryzen_dict:
        drops = check_product_page(get_soup(product_url.format(product=ryzen_dict[gpu][0])), ryzen_dict[gpu][0], 2, ryzen_dict[gpu][1], drops)
        time.sleep(1)
    return drops

In [13]:
driver = chrome_driver()

In [14]:
def check_all_products(drops=[]):
    drops = check_xbox_consoles(drops)
    drops = check_ps5_consoles(drops)
    drops = check_rtx_3000_series(drops)
    drops = check_radeon_rx_series(drops)
    drops = check_nintendo_switches(drops)
    drops = check_ryzen_series(drops)
    return drops

In [15]:
all_drops = check_all_products()

In [16]:
print(all_drops)

[['6364253', 'Nintendo - Switch 32GB Console - Gray Joy-Con', 2, 3, 3, '299.99', 'https://api.bestbuy.com/click/-/6364253/pdp', '2021/05/23 11:54:55', 'https://api.bestbuy.com/click/-/6364253/cart', False], ['6401728', 'Nintendo - Switch - Animal Crossing: New Horizons Edition 32GB Console - Multi', 2, 3, 3, '299.99', 'https://api.bestbuy.com/click/-/6401728/pdp', '2021/05/23 11:54:55', 'https://api.bestbuy.com/click/-/6401728/cart', False], ['6460946', 'Nintendo - Switch 32GB Lite - Blue', 2, 3, 3, '199.99', 'https://api.bestbuy.com/click/-/6460946/pdp', '2021/05/23 11:54:55', 'https://api.bestbuy.com/click/-/6460946/cart', False], ['51ac336757a4', 'Package - Nintendo - Switch 32GB Lite - Gray and Animal Crossing: New Horizons', 2, 3, 3, '259.98', '/site/combo/nintendo-switch-consoles/e0d0be54-459e-4ea8-900e-51ac336757a4', '2021/05/23 11:54:55', '', True], ['3f9814562e56', 'Package - Nintendo - Switch 32GB Lite - Yellow and Animal Crossing: New Horizons', 2, 3, 3, '259.98', '/site/com

In [17]:
driver.close()